In [1]:
!python --version

Python 3.12.12


In [3]:
!git clone https://github.com/RussianNLP/DRAGON.git

Cloning into 'DRAGON'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 463 (delta 110), reused 178 (delta 80), pack-reused 233 (from 1)
Receiving objects: 100% (463/463), 42.16 MiB | 38.83 MiB/s, done.
Resolving deltas: 100% (193/193), done.


In [2]:
!ls -al

total 12
drwxr-xr-x 3 root root 4096 Feb 25 18:51 .
drwxr-xr-x 5 root root 4096 Feb 25 18:51 ..
drwxr-xr-x 2 root root 4096 Feb 25 18:51 .virtual_documents


In [4]:
!pwd

/kaggle/working


In [5]:
!pip install -q "datasets" "langchain<1.0" "langchain-community<0.4" "langchain-core<1.0" "langchain-huggingface<1.0" "numpy" "tqdm" "torch>=2.1.0" "transformers" "vllm>=0.7.3" "langchain_chroma<1.0" "rouge_score" "razdel" "rag-bench>=0.2.12"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 98.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 45.7 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 333.2 k

In [8]:
import json
import random
import os

os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"

from datasets import load_dataset
from langchain_community.llms import VLLM
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer

# from rag_bench import baseline, data, evaluator, results
from DRAGON.lib.src.rag_bench import baseline, data, evaluator, results

In [9]:
torch.cuda.is_available()

True

In [10]:
HIST_PRIVATE_QA_REPO_ID: str = "ai-forever/hist-rag-bench-private-qa"
HIST_PRIVATE_TEXTS_REPO_ID: str = "ai-forever/hist-rag-bench-private-texts"
RANDOM_SEED: int = 42
EMBEDDER_NAME: str = "ai-forever/FRIDA"
LLM_NAME: str = "bond005/meno-tiny-0.1"

In [11]:
LLM_PROMPT: str = """Проанализируйте заданный контекст и ответьте на вопрос пользователя на основе сведений, предоставленных в этом контексте.
Не давайте никаких объяснений и пояснений к своему ответу. Не пишите ничего лишнего. Не извиняйтесь, не стройте диалог. Выдавайте только ответ и ничего больше.
Отвечайте на русском языке.
Если в заданном контексте нет информации для ответа на вопрос пользователя, то ничего не придумывайте и просто откажитесь отвечать.
"""
print(LLM_PROMPT)

Проанализируйте заданный контекст и ответьте на вопрос пользователя на основе сведений, предоставленных в этом контексте.
Не давайте никаких объяснений и пояснений к своему ответу. Не пишите ничего лишнего. Не извиняйтесь, не стройте диалог. Выдавайте только ответ и ничего больше.
Отвечайте на русском языке.
Если в заданном контексте нет информации для ответа на вопрос пользователя, то ничего не придумывайте и просто откажитесь отвечать.



In [12]:
def get_private_qa_dataset(version):
    return load_dataset(HIST_PRIVATE_QA_REPO_ID, revision=version)


def get_private_texts_dataset(version):
    return load_dataset(HIST_PRIVATE_TEXTS_REPO_ID, revision=version)


def get_public_to_private_texts_mapping(version):
    private_texts_ds = get_private_texts_dataset(version)
    mapping = {}
    for item in private_texts_ds["train"]:
        mapping[item["public_id"]] = item["id"]
    return mapping

In [13]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [14]:
#get public datasets (history ones)
texts_ds, questions_ds, version = data.get_datasets(is_hist=True)
print(f"version = {version}")

Latest texts version: 1.15.0
Latest questions version: 1.15.0


README.md:   0%|          | 0.00/374 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/793k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/542 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/377 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Loaded texts dataset with 542 texts
Loaded questions dataset with 600 questions
version = 1.15.0


In [15]:
#get private datasets (history ones)
qa_dataset = get_private_qa_dataset(version)
mapping = get_public_to_private_texts_mapping(version)

README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/646k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/423 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/542 [00:00<?, ? examples/s]

In [ ]:
# Kaggle FIX
import os                                                                                                                                              
# Добавляем путь с libcuda.so в LIBRARY_PATH (используется линкером при JIT-компиляции)                                                                
os.environ["LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LIBRARY_PATH", "")                                                           
print("LIBRARY_PATH =", os.environ["LIBRARY_PATH"])

LIBRARY_PATH = /usr/local/nvidia/lib64:/usr/local/cuda/lib64/stubs


In [ ]:
llm = VLLM(
    model=LLM_NAME,
    tensor_parallel_size=1,  # используем оба GPU
    max_new_tokens=256,
    top_p=0.95,
    temperature=0.3,
    vllm_kwargs={
        "gpu_memory_utilization": 0.45,
        "max_num_batched_tokens": 8192,
        "max_model_len": 4096,
        "disable_log_stats": True,
        "seed": RANDOM_SEED
    },
    disable_log_stats=True,
)
tok = AutoTokenizer.from_pretrained(LLM_NAME)

2026-02-25 19:25:05.714408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772047505.742555    1170 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772047505.750521    1170 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772047505.771225    1170 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772047505.771256    1170 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772047505.771259    1170 computation_placer.cc:177] computation placer alr

(EngineCore_DP0 pid=1170) ERROR 02-25 19:25:19 [fa_utils.py:86] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8


(EngineCore_DP0 pid=1170) <frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
(EngineCore_DP0 pid=1170) <frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.89s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.89s/it]
(EngineCore_DP0 pid=1170) 
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:03<00:00, 13.97it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:02<00:00, 13.66it/s]


In [36]:
llm

VLLM(model='bond005/meno-tiny-0.1', temperature=0.3, top_p=0.95, max_new_tokens=256, vllm_kwargs={'gpu_memory_utilization': 0.45, 'max_num_batched_tokens': 8192, 'max_model_len': 4096, 'disable_log_stats': True, 'seed': 42}, client=Qwen2ForCausalLM()(
  (model): Qwen2Model()(
    (embed_tokens): VocabParallelEmbedding(num_embeddings=151936, embedding_dim=1536, org_vocab_size=151936, num_embeddings_padded=151936, tp_size=1)
    (layers): ModuleList()(
      (0-27): 28 x Qwen2DecoderLayer()(
        (self_attn): Qwen2Attention()(
          (qkv_proj): QKVParallelLinear(in_features=1536, output_features=2048, bias=True, tp_size=1, gather_output=False)
          (o_proj): RowParallelLinear(in_features=1536, output_features=1536, bias=False, tp_size=1, reduce_results=True)
          (rotary_emb): RotaryEmbedding(head_size=128, rotary_dim=128, max_position_embeddings=32768, base=1000000.0, is_neox_style=True)(
            (apply_rotary_emb): ApplyRotaryEmb(is_neox_style=True, enable_fp32_com

In [37]:
embedding_model = HuggingFaceEmbeddings(
    model_name="ai-forever/FRIDA",
    model_kwargs={"trust_remote_code": True},
    encode_kwargs={"batch_size": 16, "prompt": "search_document: "},
    query_encode_kwargs={"prompt": "search_query: "}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/823 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [38]:
# HYBRID_RERANK = False
# RERANK_CANDIDATE_K = 20

retrieval = baseline.init_retriever(
    texts_ds,
    embedding_model,
    top_k=5,
    chunk_size=500,
    chunk_overlap=100,
    # hybrid_rerank=HYBRID_RERANK,
    # rerank_candidate_k=RERANK_CANDIDATE_K,
)
generation = baseline.init_generation(retrieval, llm, tok, system_prompt=LLM_PROMPT)


Initializing retriever


> Creating vector store
> Vector store created for 542 documents
> Done for 117.34 seconds
Initializing generation chain
> Done


In [ ]:
res = baseline.get_results(
    generation, questions_ds, write_logs=True
)

In [40]:
#saving the baseline results

res_path = "./test.json"
results.save(res, res_path)

In [ ]:
demo_public_id = 0
print(questions_ds["train"].filter(lambda it: it["id"] == demo_public_id)[0])
print('\n' + json.dumps(res[demo_public_id], ensure_ascii=False, indent=4))

Filter:   0%|          | 0/600 [00:00<?, ? examples/s]

{'id': 0, 'question': 'Какой регион был определён в качестве приоритетного для Дональда Трампа?'}

{
    "found_ids": [
        197,
        134,
        399,
        115,
        458
    ],
    "model_answer": "Индхо-Тихоокеанский регион"
}


In [ ]:
#calculate metrics

evaluation_results = evaluator.evaluate_rag_results(res, qa_dataset, mapping)

In [ ]:
_ = evaluation_results.to_table(overall_only=True)

Retrieval Metrics:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| Hit Rate |  0.7997 |
+----------+---------+
| MRR      |  0.7893 |
+----------+---------+

Generation Metrics:
+-----------------+---------+
| Metric          |   Value |
+=================+=========+
| ROUGE-1         |  0.4997 |
+-----------------+---------+
| ROUGE-2         |  0.3103 |
+-----------------+---------+
| ROUGE-L         |  0.4773 |
+-----------------+---------+
| Exact Match     |  0.1250 |
+-----------------+---------+
| Substring Match |  0.1467 |
+-----------------+---------+


In [ ]:
_ = evaluation_results.to_table(overall_only=False)

Retrieval Metrics:
+----------+-----------+--------+--------+--------+----------+
| Metric   |   Overall |   cond |     mh |    set |   simple |
+==========+===========+========+========+========+==========+
| Hit Rate |    0.7997 | 0.7733 | 0.7756 | 0.8033 |   0.8467 |
+----------+-----------+--------+--------+--------+----------+
| MRR      |    0.7893 | 0.7999 | 0.7730 | 0.7841 |   0.8000 |
+----------+-----------+--------+--------+--------+----------+

Generation Metrics:
+-----------------+-----------+--------+--------+--------+----------+
| Metric          |   Overall |   cond |     mh |    set |   simple |
+=================+===========+========+========+========+==========+
| ROUGE-1         |    0.4997 | 0.6067 | 0.4938 | 0.4180 |   0.4804 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-2         |    0.3103 | 0.3708 | 0.2920 | 0.2126 |   0.3656 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-L         |    0.4773

## DeReC Verifier (optional)
Трехзонная логика поверх confidence verifier:
- `score < LOW` -> `abstain`
- `LOW <= score < HIGH` -> `rewrite/keep/abstain` (настраивается)
- `score >= HIGH` -> `keep`

Есть опция калибровки `LOW/HIGH` на dev-срезе DRAGON по выбранной метрике.


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# USE_DEREC_VERIFIER = True
# DEREC_VERIFIER_REPO = "Makson4ic/derec-rawfc-deberta-frida"  # например: "username/derec-rawfc-deberta-frida"
# HF_TOKEN = ""  # для private repo

# DEREC_LOW_THRESHOLD = 0.15
# DEREC_HIGH_THRESHOLD = 0.3
# DEREC_MEDIUM_ACTION = "rewrite"  # "rewrite" | "keep" | "abstain"
# DEREC_CALIBRATE = True
# DEREC_DEV_SIZE = 100
# DEREC_CALIBRATION_METRIC = "exact_match"  # exact_match | substring_match | rougeL
# ABSTAIN_TEXT = "не хватает данных в контексте"


# def _build_question_index(questions_ds):
#     return {str(item["id"]): item for item in questions_ds["train"]}


# def _build_text_index(texts_ds):
#     return {item["id"]: item["text"] for item in texts_ds["train"]}


# def _build_answer_map(qa_dataset):
#     return {str(item["public_id"]): item["answer"] for item in qa_dataset["train"]}


# def _build_rewrite_prompt(tokenizer, question, evidence, answer):
#     messages = [
#         {
#             "role": "system",
#             "content": "Перепиши ответ так, чтобы остались только факты, подтверждаемые контекстом. Если контекста недостаточно, верни точную фразу: не хватает данных в контексте.",
#         },
#         {
#             "role": "user",
#             "content": (
#                 f"Контекст:\n{evidence}\n\n"
#                 f"Вопрос:\n{question}\n\n"
#                 f"Исходный ответ:\n{answer}\n\n"
#                 "Сформулируй финальный ответ одной короткой строкой."
#             ),
#         },
#     ]
#     return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


# def _rewrite_answer(question, evidence, answer):
#     return llm.invoke(_build_rewrite_prompt(tok, question, evidence, answer)).strip()


# def _score_answer(metric_name, generated, reference):
#     e = evaluator.RAGEvaluator()
#     scores = e.evaluate_generation(generated_answer=generated, reference_answer=reference)
#     return float(scores[metric_name])


# def _choose_answer(keep_answer, rewritten_answer, score, low, high, medium_action, abstain_text):
#     if score < low:
#         return abstain_text
#     if score >= high:
#         return keep_answer
#     if medium_action == "abstain":
#         return abstain_text
#     if medium_action == "keep":
#         return keep_answer
#     return rewritten_answer


# def apply_derec_verifier_three_zone(res, qa_dataset, texts_ds):
#     res = res.copy()
#     qa_dataset = qa_dataset.copy()
#     texts_ds = texts_ds.copy()
#     verifier_tok = AutoTokenizer.from_pretrained(DEREC_VERIFIER_REPO, token=(HF_TOKEN.strip() or None))
#     verifier = AutoModelForSequenceClassification.from_pretrained(DEREC_VERIFIER_REPO, token=(HF_TOKEN.strip() or None))
#     verifier.eval()
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     verifier.to(device)

#     q_idx = _build_question_index(questions_ds)
#     t_idx = _build_text_index(texts_ds)
#     answer_map = _build_answer_map(qa_dataset)

#     meta = {}
#     with torch.no_grad():
#         for qid, pred in res.items():
#             qid_str = str(qid)
#             question = q_idx.get(qid_str, {}).get("question", "")
#             answer = pred.get("model_answer", "")
#             evidence = "\n".join([t_idx.get(doc_id, "") for doc_id in pred.get("found_ids", [])])[:4000]
#             text = f"claim: question: {question}\nanswer: {answer} [SEP] evidence: {evidence}"
#             inp = verifier_tok(text, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
#             inp = {k: v.to(device) for k, v in inp.items()}
#             probs = torch.softmax(verifier(**inp).logits, dim=-1)[0].cpu().numpy()
#             true_score = float(probs[2] if probs.shape[0] >= 3 else probs.max())
#             meta[qid_str] = {
#                 "true_score": true_score,
#                 "question": question,
#                 "evidence": evidence,
#             }

#     if DEREC_MEDIUM_ACTION == "rewrite":
#         print("Preparing rewritten answers for medium-confidence zone...")
#         for qid, pred in res.items():
#             qid_str = str(qid)
#             meta[qid_str]["rewritten_answer"] = _rewrite_answer(
#                 meta[qid_str]["question"],
#                 meta[qid_str]["evidence"],
#                 pred.get("model_answer", ""),
#             )

#     low, high = DEREC_LOW_THRESHOLD, DEREC_HIGH_THRESHOLD
#     if DEREC_CALIBRATE:
#         low_grid = np.arange(0.15, 0.81, 0.05)
#         high_grid = np.arange(0.35, 0.96, 0.05)
#         dev_ids = [str(item["id"]) for item in questions_ds["train"][: min(DEREC_DEV_SIZE, len(questions_ds["train"]))]]

#         def _get_pred(qid):
#             if qid in res:
#                 return res[qid]
#             try:
#                 return res.get(int(qid))
#             except Exception:
#                 return None

#         best_score, best_low, best_high = -1.0, low, high
#         for l in low_grid:
#             for h in high_grid:
#                 if h <= l:
#                     continue
#                 vals = []
#                 for qid in dev_ids:
#                     pred = _get_pred(qid)
#                     if pred is None or qid not in answer_map or qid not in meta:
#                         continue
#                     cand = _choose_answer(
#                         keep_answer=pred.get("model_answer", ""),
#                         rewritten_answer=meta[qid].get("rewritten_answer", pred.get("model_answer", "")),
#                         score=meta[qid]["true_score"],
#                         low=l,
#                         high=h,
#                         medium_action=DEREC_MEDIUM_ACTION,
#                         abstain_text=ABSTAIN_TEXT,
#                     )
#                     vals.append(_score_answer(DEREC_CALIBRATION_METRIC, cand, answer_map[qid]))
#                 avg = float(np.mean(vals)) if vals else 0.0
#                 if avg > best_score:
#                     best_score, best_low, best_high = avg, float(l), float(h)
#         low, high = best_low, best_high
#         print(f"Calibrated thresholds: low={low:.2f}, high={high:.2f}, {DEREC_CALIBRATION_METRIC}={best_score:.4f}")

#     kept = rewritten = abstained = 0
#     for qid, pred in res.items():
#         qid_str = str(qid)
#         chosen = _choose_answer(
#             keep_answer=pred.get("model_answer", ""),
#             rewritten_answer=meta[qid_str].get("rewritten_answer", pred.get("model_answer", "")),
#             score=meta[qid_str]["true_score"],
#             low=low,
#             high=high,
#             medium_action=DEREC_MEDIUM_ACTION,
#             abstain_text=ABSTAIN_TEXT,
#         )
#         if chosen == ABSTAIN_TEXT:
#             abstained += 1
#         elif chosen == pred.get("model_answer", ""):
#             kept += 1
#         else:
#             rewritten += 1
#         pred["model_answer"] = chosen

#     print(f"DeReC verifier: kept={kept}, rewritten={rewritten}, abstained={abstained}, total={len(res)}")
#     print(f"Effective thresholds: low={low:.2f}, high={high:.2f}")
#     return res


In [2]:
1

1

In [1]:
# if USE_DEREC_VERIFIER and DEREC_VERIFIER_REPO.strip():
#     res_derec_2 = apply_derec_verifier_three_zone(res, qa_dataset, texts_ds)
# else:
#     print("DeReC verifier is disabled")

In [76]:
# def _build_question_index(questions_ds):
#     return {str(item["id"]): item for item in questions_ds["train"]}

# def _build_text_index(texts_ds):
#     return {item["id"]: item["text"] for item in texts_ds["train"]}

# def apply_derec_verifier(rag_results, questions_ds, texts_ds, repo_id, threshold=0.5, hf_token=None, max_evidence_chars=4000):
#     rag_results = rag_results.copy()
#     questions_ds = questions_ds.copy()
#     texts_ds = texts_ds.copy()
#     tokenizer = AutoTokenizer.from_pretrained(repo_id, token=hf_token)
#     model = AutoModelForSequenceClassification.from_pretrained(repo_id, token=hf_token)
#     model.eval()
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     model.to(device)

#     question_idx = _build_question_index(questions_ds)
#     text_idx = _build_text_index(texts_ds)

#     abstained = 0
#     total = 0
#     with torch.no_grad():
#         for qid, pred in rag_results.items():
#             total += 1
#             q = question_idx.get(str(qid), {})
#             question = q.get("question", "")
#             answer = pred.get("model_answer", "")
#             found_ids = pred.get("found_ids", [])

#             evidence = "\n".join([text_idx.get(doc_id, "") for doc_id in found_ids])[:max_evidence_chars]
#             claim = f"question: {question}\nanswer: {answer}"
#             text = f"claim: {claim} [SEP] evidence: {evidence}"

#             inputs = tokenizer(text, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
#             inputs = {k: v.to(device) for k, v in inputs.items()}
#             probs = torch.softmax(model(**inputs).logits, dim=-1)[0].cpu().numpy()
#             print(probs)

#             true_score = float(probs[2] if probs.shape[0] >= 3 else probs.max())
#             print(true_score, threshold)
#             if true_score < threshold:
#                 pred["model_answer"] = ABSTAIN_TEXT
#                 abstained += 1

#     print(f"DeReC verifier: abstained {abstained}/{total} ({abstained/max(total,1):.2%})")
#     return rag_results

In [27]:
# res

In [64]:
# if USE_DEREC_VERIFIER and DEREC_VERIFIER_REPO.strip():
#     res_derec = apply_derec_verifier(
#         res, questions_ds, texts_ds,
#         repo_id=DEREC_VERIFIER_REPO.strip(),
#         threshold=DEREC_VERIFIER_THRESHOLD,
#         hf_token=(HF_TOKEN.strip() or None),
#     )
# else:
#     print("DeReC verifier is disabled")

In [ ]:
res_derec

In [37]:
#saving the baseline results

res_path_derec = "./test_derec.json"
results.save(res_derec, res_path_derec)

In [77]:
demo_public_id = 0
print(questions_ds["train"].filter(lambda it: it["id"] == demo_public_id)[0])
print('\n' + json.dumps(res_derec_2[demo_public_id], ensure_ascii=False, indent=4))

{'id': 0, 'question': 'Какой регион был определён в качестве приоритетного для Дональда Трампа?'}

{
    "found_ids": [
        197,
        134,
        399,
        115,
        458
    ],
    "model_answer": "Индхо-Тихоокеанский регион"
}


In [78]:
#calculate metrics

evaluation_results_derec = evaluator.evaluate_rag_results(res_derec_2, qa_dataset, mapping)

In [ ]:
_ = evaluation_results.to_table(overall_only=True)

Retrieval Metrics:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| Hit Rate |  0.7997 |
+----------+---------+
| MRR      |  0.7893 |
+----------+---------+

Generation Metrics:
+-----------------+---------+
| Metric          |   Value |
+=================+=========+
| ROUGE-1         |  0.5069 |
+-----------------+---------+
| ROUGE-2         |  0.3159 |
+-----------------+---------+
| ROUGE-L         |  0.4849 |
+-----------------+---------+
| Exact Match     |  0.1233 |
+-----------------+---------+
| Substring Match |  0.1500 |
+-----------------+---------+


In [79]:
_ = evaluation_results_derec.to_table(overall_only=True)

Retrieval Metrics:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| Hit Rate |  0.7997 |
+----------+---------+
| MRR      |  0.7893 |
+----------+---------+

Generation Metrics:
+-----------------+---------+
| Metric          |   Value |
+=================+=========+
| ROUGE-1         |  0.4903 |
+-----------------+---------+
| ROUGE-2         |  0.2945 |
+-----------------+---------+
| ROUGE-L         |  0.4675 |
+-----------------+---------+
| Exact Match     |  0.1183 |
+-----------------+---------+
| Substring Match |  0.1450 |
+-----------------+---------+


In [69]:
_ = evaluation_results_derec.to_table(overall_only=False)

Retrieval Metrics:
+----------+-----------+--------+--------+--------+----------+
| Metric   |   Overall |   cond |     mh |    set |   simple |
+==========+===========+========+========+========+==========+
| Hit Rate |    0.7997 | 0.7733 | 0.7756 | 0.8033 |   0.8467 |
+----------+-----------+--------+--------+--------+----------+
| MRR      |    0.7893 | 0.7999 | 0.7730 | 0.7841 |   0.8000 |
+----------+-----------+--------+--------+--------+----------+

Generation Metrics:
+-----------------+-----------+--------+--------+--------+----------+
| Metric          |   Overall |   cond |     mh |    set |   simple |
+=================+===========+========+========+========+==========+
| ROUGE-1         |    0.4903 | 0.5921 | 0.5155 | 0.4416 |   0.4121 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-2         |    0.2945 | 0.3613 | 0.2951 | 0.2198 |   0.3016 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-L         |    0.4675

In [70]:
_ = evaluation_results.to_table(overall_only=False)

Retrieval Metrics:
+----------+-----------+--------+--------+--------+----------+
| Metric   |   Overall |   cond |     mh |    set |   simple |
+==========+===========+========+========+========+==========+
| Hit Rate |    0.7997 | 0.7733 | 0.7756 | 0.8033 |   0.8467 |
+----------+-----------+--------+--------+--------+----------+
| MRR      |    0.7893 | 0.7999 | 0.7730 | 0.7841 |   0.8000 |
+----------+-----------+--------+--------+--------+----------+

Generation Metrics:
+-----------------+-----------+--------+--------+--------+----------+
| Metric          |   Overall |   cond |     mh |    set |   simple |
+=================+===========+========+========+========+==========+
| ROUGE-1         |    0.5069 | 0.6177 | 0.4885 | 0.4278 |   0.4936 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-2         |    0.3159 | 0.3795 | 0.2877 | 0.2234 |   0.3731 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-L         |    0.4849

## Top-10 Documents By Fact-Check Score
Вычисляет score класса `true` (DeReC verifier) для каждого ответа и агрегирует его по найденным документам (`found_ids`).
Выводит top-10 документов по среднему score и количеству появлений.


In [ ]:
from collections import defaultdict

TOP_N_DOCS = 10

if not (DEREC_VERIFIER_REPO and DEREC_VERIFIER_REPO.strip()):
    print("Set DEREC_VERIFIER_REPO first (cell 'DeReC Verifier (optional)').")
else:
    verifier_tok = AutoTokenizer.from_pretrained(DEREC_VERIFIER_REPO, token=(HF_TOKEN.strip() or None))
    verifier = AutoModelForSequenceClassification.from_pretrained(DEREC_VERIFIER_REPO, token=(HF_TOKEN.strip() or None))
    verifier.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    verifier.to(device)

    q_idx = _build_question_index(questions_ds)
    t_idx = _build_text_index(texts_ds)

    doc_score_sum = defaultdict(float)
    doc_score_cnt = defaultdict(int)
    doc_score_max = defaultdict(float)

    with torch.no_grad():
        for qid, pred in res.items():
            qid_str = str(qid)
            question = q_idx.get(qid_str, {}).get("question", "")
            answer = pred.get("model_answer", "")
            found_ids = pred.get("found_ids", [])
            evidence = "\n".join([t_idx.get(doc_id, "") for doc_id in found_ids])[:4000]

            text = f"claim: question: {question}\nanswer: {answer} [SEP] evidence: {evidence}"
            inp = verifier_tok(text, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
            inp = {k: v.to(device) for k, v in inp.items()}
            probs = torch.softmax(verifier(**inp).logits, dim=-1)[0].cpu().numpy()
            true_score = float(probs[2] if probs.shape[0] >= 3 else probs.max())

            for doc_id in found_ids:
                doc_score_sum[doc_id] += true_score
                doc_score_cnt[doc_id] += 1
                doc_score_max[doc_id] = max(doc_score_max[doc_id], true_score)

    ranked = []
    for doc_id, total in doc_score_sum.items():
        cnt = doc_score_cnt[doc_id]
        avg = total / cnt
        ranked.append((doc_id, avg, cnt, doc_score_max[doc_id]))

    ranked.sort(key=lambda x: (-x[1], -x[2], -x[3]))
    top_docs = ranked[:TOP_N_DOCS]

    print(f"Top-{TOP_N_DOCS} documents by average fact-check true_score")
    for i, (doc_id, avg_score, cnt, max_score) in enumerate(top_docs, 1):
        print(f"{i:02d}. doc_id={doc_id} | avg_score={avg_score:.4f} | occurrences={cnt} | max_score={max_score:.4f}")
